In [2]:
import pandas as pd
import numpy as np
import json
from datetime import date
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
artist_df = pd.read_csv('./data/artists.csv')
tracks_df = pd.read_csv('./data/tracks.csv')
with open("./data/dict_artists.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [ ]:
artist_df.drop(['id'], axis=1, inplace=True)

In [ ]:
artist_df = artist_df[artist_df['genres'] != '[]']
artist_df = artist_df.sort_values(by=['popularity'], ascending=False)
l = len(artist_df)/15
artist_df = artist_df[:round(l)]
artist_df = artist_df.reset_index(drop=True)

In [ ]:
artist_df

19907.733333333334

In [19]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(artist_df['genres'])

In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 
results = {}
for idx, row in artist_df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
    similar_items = [(cosine_similarities[idx][i], artist_df['name'][i]) for i in similar_indices] 
    results[row['name']] = similar_items[1:]

In [ ]:
def item(id):  
    return artist_df.loc[artist_df['name'] == id]['genres'].tolist()[0].split(' - ')[0] 

# Just reads the results out of the dictionary.def 
recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")    recs = results[item_id][:num]   
    for rec in recs: 
       print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")